In [1]:
#import packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer
import boto3
import requests
from sqlalchemy import create_engine

In [2]:
def has_embedded_video(file, method):
    
    video_flags = ['youtube','video','.mp4','player']
    unless = ['soundcloud']
    html_snippet = []
    
    if method == "s3":
        obj = s3.Object('vpal-link-data','files/'+file)
        soup = BeautifulSoup(obj.get()['Body'].read(),'lxml',parse_only=SoupStrainer("iframe"))
    elif method == "scrape":
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
        req = requests.get(file, headers = headers)
        page = req.content
        soup = BeautifulSoup(page, 'lxml',parse_only=SoupStrainer("iframe"))
        
    if soup.find("iframe"):
        for frame in soup.find_all("iframe"):
            for key in frame.attrs:
                if any(flag in frame.attrs[key] for flag in video_flags) and any(u not in frame.attrs[key] for u in unless):
                    html_snippet.append(frame)
    
    html_snippet = list(set(html_snippet))               
    hasit = len(html_snippet)
    
    return hasit, html_snippet

In [3]:
def has_hyperlinked_video(file, method):
    
    video_flags = ['youtube','vimeo']
    html_snippet = []
    
    if method == "s3":
        obj = s3.Object('vpal-link-data','files/'+file)
        soup = BeautifulSoup(obj.get()['Body'].read(),'lxml',parse_only=SoupStrainer("a"))
    elif method == "scrape":
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
        req = requests.get(file, headers = headers)
        page = req.content
        soup = BeautifulSoup(page, 'lxml',parse_only=SoupStrainer("a"))
        
    if soup.find("a"):
        for a in soup.find_all("a"):
            for key in a.attrs:
                if any(flag in a.attrs[key] for flag in video_flags):
                    html_snippet.append(a)
                    
    html_snippet = list(set(html_snippet))               
    hasit = len(html_snippet)
    
    return hasit, html_snippet

In [4]:
def prune_df(df):
    bad = ['user','login','pin1','calendar',
       '0%2C0%2C0%2C0%2C0%2C1','0%2C0%2C0%2C0%2C0%2C2','tagged',
       'lang=','/?', 'event/',
       '.aif','.mp3','.gif','.pdf','.gz','.wtt','.mswd','.dd','.key','.ppt','.txt',
       '.ppiv','.m','.ps~','.prev','search','ppframe.htm',
      '.fits','.tgz','.dat','.presentation','.pages','bagelsite','AirlineSite']
    
    msk = [False if any(b in i for b in bad) else True for i in df.url]
    pruned_df = df[msk]
    
    inv_dup_msk = pruned_df.duplicated(subset="checksum")
    pruned_df = pruned_df[~inv_dup_msk]
    
    return pruned_df

In [5]:
def find_all_videos(data_df,method,logging=False):
#     urls = list(data_df.url)
#     huids = list(data_df.person_id)

    if method == "s3":
        files_or_urls = data_df.id
    elif method == "scrape":
        files_or_urls = data_df.url
        
    num_embedded_videos = []
    num_hyperlinked_videos = []
    embedded_snippet = []
    hyperlinked_snippet = []
    
    total_urls = data_df.shape[0]
    
    # check for videos
    print("beginning search. ({} urls)".format(total_urls))
    
    for ind, file in enumerate(files_or_urls):
        embedded = has_embedded_video(file, method)
        hyperlinked = has_hyperlinked_video(file, method)
        
        num_embedded_videos.append(embedded[0])
        embedded_snippet.append(embedded[1])
        
        num_hyperlinked_videos.append(hyperlinked[0])
        hyperlinked_snippet.append(hyperlinked[1])
        
        if logging and total_urls >= 100:
            pct_done = (ind/total_urls)*100
            if pct_done % 10 == 0:
                print("Done with {}% of urls.".format(pct_done))
                
    print("finished searching all urls.")
    return num_embedded_videos, embedded_snippet, num_hyperlinked_videos, hyperlinked_snippet

In [14]:
# access database
s3 = boto3.resource('s3')
ssm = boto3.client('ssm',region_name='us-east-1')
db_connection_string = ssm.get_parameter(Name='/link/prod/read/db_connection_string')['Parameter']['Value']
engine = create_engine(db_connection_string)
# person_resource = pd.read_sql_table('person_resource',engine)
link_ent = pd.read_sql_table('link_entity',engine)
link_ent_resource = pd.read_sql_table('link_entity_resources',engine)
resource = pd.read_sql_table('link_resource',engine)
# person_resource.columns = ['person_id','id']
# data = person_resource.merge(resource)


In [17]:
link_ent.columns = ['entity_id','name','type']

In [21]:
link_ent_resource.head()

,id,entity_id,resource_id
0,1,fe299ae2-6e37-417f-8544-245b1c37dc58,cde57122f81e1972926c488220ec55981b80324e
1,2,fe299ae2-6e37-417f-8544-245b1c37dc58,b1bded44eab335a615058bdc477746d2dd08b36b
2,3,fe299ae2-6e37-417f-8544-245b1c37dc58,9e3e28717985df55090a6288e260ddc09f04fb68
3,4,fe299ae2-6e37-417f-8544-245b1c37dc58,0a9cc22f7b17f3f99d971dec60e0f2efa9608b5c
4,5,fe299ae2-6e37-417f-8544-245b1c37dc58,936fa927a34af1c4a6fcdae587ef64b66428f44f


In [27]:
resource.columns = ['resource_id','url','content_type','checksum']

In [23]:
data = link_ent.merge(link_ent_resource.drop('id',axis=1))

In [28]:
data = data.merge(resource)

In [43]:
data.columns = ['entity_id', 'name', 'type', 'id', 'url', 'content_type',
       'checksum']

In [44]:
not_people = data[data.type != 'person']

In [45]:
not_people.head()

,entity_id,name,type,id,url,content_type,checksum
0,f38b5160-e570-4995-8fe2-1d69d37bfc30,Harvard Library,,d2420be2f5bee0221bc3742ad2cac3fe9c9404fa,https://library.harvard.edu/onlineexhibits/sol...,text/html,cbd9e687d0fbc07db52396d302eeed1f
1,f38b5160-e570-4995-8fe2-1d69d37bfc30,Harvard Library,,13cf1a93e12244d946f089dea724e743c8f8a48b,https://library.harvard.edu/collections/epj/ja...,text/html,e0ee9f82e59efea5371bbe2e1a527f57
2,f38b5160-e570-4995-8fe2-1d69d37bfc30,Harvard Library,,c2ef2a6af9837f28078504c4d1f88aed67712844,https://library.harvard.edu/collections/ipcc/a...,text/html,ff4e781025eb31f2f79799cbe8a97090
3,f38b5160-e570-4995-8fe2-1d69d37bfc30,Harvard Library,,c8b9adedf487df9ef7b96fc9be0235c59f870801,https://library.harvard.edu/collections/record...,text/html; charset=UTF-8,63ac61ac59828ed999576f4ca17e4f63
4,f38b5160-e570-4995-8fe2-1d69d37bfc30,Harvard Library,,11cd7d9b4b2ce6166ce4a910e9da5e72397bd84e,https://library.harvard.edu/libraries/yenching,text/html; charset=UTF-8,56ea2856bc58cf8d3f418951ff8f6c41


In [36]:
file = not_people.resource_id[0]
obj = s3.Object('vpal-link-data','files/'+file)
soup = BeautifulSoup(obj.get()['Body'].read(),'lxml')
url = not_people.url[0]
url

'https://library.harvard.edu/onlineexhibits/solti/twentieth/bartok-sonata/index.html'

In [39]:
entity_ids = not_people.entity_id.unique()

In [57]:
entity_dict = {}

for i in entity_ids:
    
    temp_df = prune_df(not_people[not_people.entity_id ==i])
    temp_vids = find_all_videos(temp_df, method = "s3")
    temp_df['num_embedded'] = temp_vids[0]
    temp_df['embedded_html'] = temp_vids[1]
    temp_df['num_hyperlinked'] = temp_vids[2]
    temp_df['hyperlinked_html'] = temp_vids[3]
    temp_df['total_videos'] = temp_df.num_embedded + temp_df.num_hyperlinked
    temp_df = temp_df[['entity_id','url','total_videos','num_embedded','embedded_html','num_hyperlinked','hyperlinked_html']]
    
    entity_dict[i] = temp_df
    del temp_df


beginning search. (401 urls)
finished searching all urls.
beginning search. (162 urls)
finished searching all urls.
beginning search. (70 urls)
finished searching all urls.
beginning search. (22 urls)
finished searching all urls.
beginning search. (149 urls)
finished searching all urls.
beginning search. (512 urls)
finished searching all urls.
beginning search. (123 urls)
finished searching all urls.
beginning search. (15 urls)
finished searching all urls.
beginning search. (134 urls)
finished searching all urls.
beginning search. (13 urls)
finished searching all urls.
beginning search. (116 urls)
finished searching all urls.
beginning search. (113 urls)
finished searching all urls.
beginning search. (887 urls)
finished searching all urls.
beginning search. (887 urls)
finished searching all urls.
beginning search. (104 urls)
finished searching all urls.
beginning search. (128 urls)
finished searching all urls.
beginning search. (641 urls)
finished searching all urls.
beginning search. 

/Users/agp234/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.cfa.harvard.edu/dvl/special/pubaff/20070920.rm
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


finished searching all urls.
beginning search. (126 urls)
finished searching all urls.
beginning search. (5 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginning search. (130 urls)
finished searching all urls.
beginning search. (137 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginning search. (0 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginning search. (11 urls)
finished searching all urls.
beginning search. (481 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginning search. (608 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginning search. (0 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginning search. (8 urls)
finished searching all urls.
beginning search. (1 urls)
finished searching all urls.
beginnin

In [58]:
# entity_dict['f38b5160-e570-4995-8fe2-1d69d37bfc30'][entity_dict['f38b5160-e570-4995-8fe2-1d69d37bfc30']['total_videos'] > 0]

In [59]:
final_entity_data = pd.concat([df for df in entity_dict.values()], ignore_index=True)

In [56]:
not_people.shape

(75635, 7)

In [60]:
final_entity_data.to_csv('data/entity_videos.csv')

In [22]:
# person bio
# data = pd.read_csv('data/person_resource.csv')

In [38]:
fac = 40809295

In [51]:
faculty_dict = {}
faculty_ids = data.person_id.unique()
file = open("log.txt","w")

for i in faculty_ids:
    
    file.write(i)
    temp_df = prune_df(data[data.person_id ==i])
    try:
        temp_vids = find_all_videos(temp_df, method = "scrape")
    except 
    temp_df['num_embedded'] = temp_vids[0]
    temp_df['embedded_html'] = temp_vids[1]
    temp_df['num_hyperlinked'] = temp_vids[2]
    temp_df['hyperlinked_html'] = temp_vids[3]
    temp_df['total_videos'] = temp_df.num_embedded + temp_df.num_hyperlinked
    temp_df = temp_df[['person_id','url','total_videos','num_embedded','embedded_html','num_hyperlinked','hyperlinked_html']]
    
    faculty_dict[i] = temp_df
    del temp_df
    file.write("\n")

file.close()

beginning search. (318 urls)
finished searching all urls.


In [9]:
faculty_dict = {}
faculty_ids = data.person_id.unique()
file = open("log.txt","w") 
for i in faculty_ids:
    file.write(i)
    temp_df = prune_df(data[data.person_id == i])
    temp_vids = find_all_videos(temp_df, method = "scrape")
    temp_df['num_videos'] = temp_vids
    temp_df = temp_df[['person_id','url','num_videos']]
    
    faculty_dict[i] = temp_df
    del temp_df
file.close()

In [16]:
dfx = pd.concat([df for df in faculty_dict.values()], ignore_index=True)

In [18]:
dfx.to_csv('data/partiallycomplete.csv')

In [11]:
# data.to_csv('data/person_resource.csv')

In [10]:
# videos = find_all_videos(data,logging=True)
# data['num_videos'] = videos
# final_df = data[['person_id','url','num_videos']]

In [11]:
faculty_ids = data.person_id.unique()
[i for i,j in enumerate(faculty_ids) if j == 40448225]

[258]

In [57]:
for i in data.person_id.unique():
    if prune_df(data[data.person_id == i]).shape[0] == 741:
        print(i)

40652299


In [58]:
data[data.person_id == 40652299]

,Unnamed: 0,person_id,id,url,content_type,checksum
28795,28795,40652299,7041d641c3c541ffa2e352903ad93177571a96ed,https://vcg.seas.harvard.edu/,text/html,8dd1be5b89c66471d619aa051ffcd96d
28796,28796,40652299,c225045beb4e8b2c1805d33727571b2c755b9f47,https://vcg.seas.harvard.edu/publications/comm...,application/x-bibtex; charset=UTF-8,5773ac71b2b84174a0d8ce437e3b43f4
28797,28797,40652299,000bca766d36a3e3bf418e7520be1bf0df8733bf,https://vcg.seas.harvard.edu/publications/comm...,text/html,29f2acc08562d92c1c3596f04bbe73bf
28798,28798,40652299,2871f778c22c41093795e0ece2745c9ff9960c5e,https://vcg.seas.harvard.edu/publications/seq2...,application/x-bibtex; charset=UTF-8,2696aefd0aac96008a560f3bf5d9bfb6
28799,28799,40652299,f685f80d1aa38ec128d8630d392a5f447a918503,https://vcg.seas.harvard.edu/made-with,text/html,c6c40e252830013031d38e605338f21c
28800,28800,40652299,61b6a12e1036ef86274276ee6e7c235c3d55e78e,https://vcg.seas.harvard.edu/publications/seq2...,text/html,b85220e3b1d9f71a9ffe86349709ec5b
28801,28801,40652299,d1f55589708048a8e330e7d14ebdea9cb60ad81a,https://vcg.seas.harvard.edu/publications/hybr...,application/x-bibtex; charset=UTF-8,d6a7e7ce5cea6ea8ebb40662b1e69eea
28802,28802,40652299,96a1a8193f74bec4cd55d7f0c1b131377fe0193f,https://vcg.seas.harvard.edu/publications/comm...,application/pdf; charset=UTF-8,b32e489b1917bd770be848bec44983ca
28803,28803,40652299,2667616b40104d00b227ed3a5580f5ff9aae86ca,https://vcg.seas.harvard.edu/publications/hybr...,text/html,582bbfd3e683ba7209604c3ab936846b
28804,28804,40652299,d76ce1adc5daa012fd78a11cf5d4c9d7647e7549,https://vcg.seas.harvard.edu/publications/eval...,text/html,f168d7c07114aef85176934f91916fa7


In [138]:
import csv

with open('csvfile.csv','w',newline='') as csvheader:
    spamwriter = csv.writer(csvheader, delimiter=',')
    spamwriter.writerow(data.columns)

In [139]:
with open('csvfile.csv','a',newline='') as csvfile:
    spamwriter2 = csv.writer(csvfile, delimiter=',')
    spamwriter2.writerows((data.iloc[i] for i  in range(data.iloc[11:14].shape[0])))

In [140]:
test = pd.read_csv('data/error_at_90810898.csv')

In [61]:
link_ent.to_csv('data/link_entity.csv')